In [ ]:
# https://www.kaggle.com/datasets/abdelhamidtebba/booksrecommendedbyworldfamouspersonnalities/data

import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('./books.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# drop na values
df = df.dropna()
df.shape

In [ ]:
#conversion to lowercase
df['recommender'] = df['recommender'].str.lower().str.replace(" ","").str.replace("|",' ')
df['author'] = df['author'].str.lower()
df['category'] = df['category'].str.lower()

In [ ]:
# Find duplicated rows
df[df.duplicated(['title'], keep = False)] 

In [ ]:
#Our Data after now looks like
df.head()

In [ ]:
def create_content(x):
    return ''.join(x['title']) + " " + ''.join(x['recommender']) + " " + ''.join(x['author']) +" "+''.join(x['category'])

df['content'] = df.apply(create_content, axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['content'])

count_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
#Construct a reverse map of indices and movie titles

indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recommendations(title, cosine_sim = cosine_sim):
    # Get the index of the movie that matches the title
    id_ = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[id_]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    data = df['title'].iloc[book_indices]
    data.index = np.arange(1, len(data)+1)
    data = pd.DataFrame(data)

    return data

In [ ]:
book_name = input("Enter the book name: ")
book_name
get_recommendations(book_name.title())